In [4]:
#Step-1:importing required packges
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Y0S6hzU6gkLzXCXp1cZe997hnsJzbq_a3gdgEKJXBqb0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu-gb.cloud-object-storage.appdomain.cloud')

bucket = 'ibmhackchallenge-donotdelete-pr-zvgmqfacywlwws'
object_key = 'Placement_Data_Full_Class.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()



,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0


In [6]:
#Step-2:Know the dataset
#data=pd.read_csv('/content/Placement_Data_Full_Class.csv')

In [8]:
#Step-3 Data Preprocessing
lst=['gender','hsc_s','degree_t','workex','specialisation','status']
for label in lst:#encoding data
  data[label]=LabelEncoder().fit(data[label]).transform(data[label])

col=['ssc_p','hsc_p','workex','mba_p','status']

In [9]:
#making copies of original data set for classification and regresiion analysis
data_clf=data.copy()
data_reg=data.copy()

In [10]:
data_clf.drop(['sl_no','ssc_b','hsc_b','salary'],axis=1,inplace=True)#removing unnecessary columns
X=data_clf.drop(['status'],axis=1)
Y=data_clf['status']


In [11]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=55)




In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
rf=RandomForestClassifier()
rf.fit(X_train,Y_train)
y_pred5=rf.predict(X_test)


In [13]:
print(metrics.accuracy_score(Y_test,y_pred5)*100)

90.74074074074075


In [14]:
print(rf.predict([[ 1,67,91.00,1,58.00,2,0,55.0,1,58.80]]))

[1]


/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [15]:
import pickle
pickle.dump(rf,open("placed.pkl","wb"))

**REGRESSION**

In [16]:
data_reg.dropna(inplace=True)
x=data_reg[['gender','hsc_s','degree_t','etest_p','mba_p']]
y=data_reg['salary']

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(x)

# Save the scaler object to preprocess.pkl
with open('preprocess.pkl', 'wb') as preprocess_file:
    pickle.dump(scaler, preprocess_file)

y[y > 400000]

X_scaled = X_scaled[y < 400000]
y = y[y < 400000]



In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.25)

from sklearn.linear_model import LinearRegression
reg=LinearRegression()
reg.fit(x_train,y_train)
y_predict=reg.predict(x_test)

In [18]:
answer=reg.predict([[1,1,1,0.75,0.39592317]])
print(answer)

[279134.3046332]


In [19]:
with open('model.pkl', 'wb') as model_file:
    pickle.dump(reg, model_file)

In [20]:
best_model = rf

In [21]:
best_model

RandomForestClassifier()

In [22]:
!pip install -U ibm-watson-machine-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: ibm-watson-machine-learning
    Found existing installation: ibm-watson-machine-learning 1.0.311
    Uninstalling ibm-watson-machine-learning-1.0.311:
      Successfully uninstalled ibm-watson-machine-learning-1.0.311


In [24]:
from ibm_watson_machine_learning import APIClient
import json
import numpy as np

In [25]:
wml_credentials = {
    "apikey":"maz1CYxFN7uzSp5yHUsWxynagbwvrSHMeGjMhDGlBS_j",
    "url":"https://eu-gb.ml.cloud.ibm.com"
}

In [29]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ------------------------------  ------------------------
ID                                    NAME                            CREATED
22dcc299-8f7f-4dd2-9a2e-7ca88fab29d4  ibm_hackchallenge_DEPLOYMENT_1  2023-08-19T06:10:22.765Z
------------------------------------  ------------------------------  ------------------------


,ID,NAME,CREATED
0,22dcc299-8f7f-4dd2-9a2e-7ca88fab29d4,ibm_hackchallenge_DEPLOYMENT_1,2023-08-19T06:10:22.765Z


In [30]:
SPACE_ID="22dcc299-8f7f-4dd2-9a2e-7ca88fab29d4"

In [31]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [32]:
MODEL_NAME = 'MIRACULOUS'
DEPLOYMENT_NAME = 'PP Deployment'
BEST_MODEL = best_model

In [33]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.2-py3.10')
print(software_spec_uid)

# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.1',
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
} 

#Save model
model_details = wml_client.repository.store_model(model=BEST_MODEL,meta_props=model_props, training_data=X_train,training_target=Y_train) 

b56101f1-309d-549b-a849-eaa63f77b2fb


/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/ibm_watson_machine_learning/repository.py:607: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [34]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'status',
  'schemas': {'input': [{'fields': [{'name': 'gender', 'type': 'int64'},
      {'name': 'ssc_p', 'type': 'float64'},
      {'name': 'hsc_p', 'type': 'float64'},
      {'name': 'hsc_s', 'type': 'int64'},
      {'name': 'degree_p', 'type': 'float64'},
      {'name': 'degree_t', 'type': 'int64'},
      {'name': 'workex', 'type': 'int64'},
      {'name': 'etest_p', 'type': 'float64'},
      {'name': 'specialisation', 'type': 'int64'},
      {'name': 'mba_p', 'type': 'float64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': 'b56101f1-309d-549b-a849-eaa63f77b2fb',
   'name': 'runtime-22.2-py3.10'},
  'training_data_references': [{'id': '1',
    'location': {},
    'schema': {'fields': [{'name': 'gender', 'type': 'int64'},
      {'name': 'ssc_p', 'type': 'float64'},
      {'name': 'hsc_p', 'type': 'float64'},
      {'name': 'hsc_s', 'type': 'int64'},
      {'name': 'degree_p',

In [35]:
model_uid = wml_client.repository.get_model_uid(model_details);
model_uid

This method is deprecated, please use get_model_id()


'b7494d55-9798-425b-8671-e30696037078'

In [36]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}


In [37]:
# Deploy
deployment = wml_client.deployments.create(
    artifact_uid=model_uid, 
    meta_props=deployment_props 
)



#######################################################################################

Synchronous deployment creation for uid: 'b7494d55-9798-425b-8671-e30696037078' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='1983e619-86c9-4525-a075-f7fb6ef57904'
------------------------------------------------------------------------------------------------


